# Simple Wind and Solar Hybrid Plant using API Key Example
---
In this example, we will use the NREL API key to download wind and solar resource data at a given location and simulate a hybrid system.

### Import Required Modules
We start by importing the necessary modules and setting up our working environment.

In [2]:
from hopp.utilities.keys import set_nrel_key_dot_env
from hopp.simulation.hybrid_simulation import HybridSimulation
from hopp.simulation.technologies.sites import SiteInfo
from hopp.simulation.technologies.sites import flatirons_site as sample_site

/Users/kbrunik/github/forked/HOPP/examples/workshop/log/hybrid_systems_2023-09-28T09.41.52.276176.log


### Set API Key
To access the Wind Toolkit (wind resource) and NSRDB (solar resource) data, we need to set an API key. You can obtain an API key from the [NREL developer website](https://developer.nrel.gov/signup/).

To set up the `NREL_API_KEY` required for resource downloads, you can create an Environment Variable called `NREL_API_KEY`. Otherwise, you can keep the key in a new file called ".env" in the root directory of this project.

Create a file ".env" that contains the single line:
`NREL_API_KEY=key`

In [3]:
# Set API key using the .env
set_nrel_key_dot_env()

#### (Optional)

This step is if you do not set the API key using a ".env" file.

In [20]:
import os
from hopp.utilities.keys import set_developer_nrel_gov_key

# Set API key manually if not using the .env
global NREL_API_KEY
NREL_API_KEY = os.getenv("NREL_API_KEY") # Set this key manually here if you are not setting it using the .env
set_developer_nrel_gov_key(NREL_API_KEY)  

### Retrieve Resource Data
Obtain the resource data for your location. Set the `site_input_data` dictionary with the desired location, year, and resources used in hybrid plant. The `SiteInfo` will download the resource data using the API key.

In [5]:
site_input_data = sample_site
site_input_data['lat'] = 35.2018863
site_input_data['lon'] = -101.945027
site_input_data['year'] = 2012

site = SiteInfo(
    site_input_data,
    solar=True,
    wind=True,
    wave=False
)

### Define System Capacities
Specify the capacities of the solar, wind, and interconnection components of the hybrid system in megawatts (MW).

In [6]:
solar_size_mw = 50
wind_size_mw = 50
interconnection_size_mw = 50

### Define Technologies Configuration
Configure the characteristics of each technology in the system, including PV (solar), wind, and grid interconnection.

In [7]:
technologies = {
    'pv': {
        'system_capacity_kw': solar_size_mw * 1000
    },
    'wind': {
        'num_turbines': 10,
        'turbine_rating_kw': 5000
    },
    'grid': {
        'interconnect_kw': interconnection_size_mw * 1000
    }
}

### Create the Hybrid Simulation Model
Create an instance of the `HybridSimulation` class by providing the technology configuration and site information.

In [8]:
hybrid_plant = HybridSimulation(technologies, site)

### Configure Simulation Parameters
Modify various parameters for the simulation, such as the system capacity, power purchase agreement (PPA) price, and DC degradation for the PV system.

In [9]:
hybrid_plant.pv.system_capacity_kw = solar_size_mw * 1000
hybrid_plant.wind.system_capacity_by_num_turbines(wind_size_mw * 1000)
hybrid_plant.ppa_price = 0.1
hybrid_plant.pv.dc_degradation = [0] * 25

### Run the Simulation
Simulate the hybrid renewable energy system for a specified number of years (in this case, 25 years).

In [10]:
hybrid_plant.simulate(25)

### Retrieve and Display Results
Access the simulation results, including annual energies, capacity factors and net present values (NPVs), and print them to the console.

In [11]:
annual_energies = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

wind_installed_cost = hybrid_plant.wind.total_installed_cost
solar_installed_cost = hybrid_plant.pv.total_installed_cost
hybrid_installed_cost = hybrid_plant.grid.total_installed_cost

print("Wind Installed Cost: {}".format(wind_installed_cost))
print("Solar Installed Cost: {}".format(solar_installed_cost))
print("Hybrid Installed Cost: {}".format(hybrid_installed_cost))

print("Wind NPV: {}".format(hybrid_plant.net_present_values.wind))
print("Solar NPV: {}".format(hybrid_plant.net_present_values.pv))
print("Hybrid NPV: {}".format(hybrid_plant.net_present_values.hybrid))

print(annual_energies)
print(cf)
print(npvs)

Wind Installed Cost: 72700000.0
Solar Installed Cost: 48000000.0
Hybrid Installed Cost: 120700000.0
Wind NPV: 54696750.69692759
Solar NPV: 59878156.04987331
Hybrid NPV: 104313009.79654434
{"pv": 104053614.17593749, "wind": 156936144.3525466, "hybrid": 250319641.2168857}
{"pv": 23.75653291688071, "wind": 35.83016994350379, "hybrid": 29.793351430192253}
{"pv": 59878156.04987331, "wind": 54696750.69692759, "hybrid": 104313009.79654434}
